In [5]:
import pandas as pd
import plotly.express as px
import MetaTrader5 as mt5

from account_credentials import LOGIN, PASSWORD, SERVER
from datetime import datetime

In [6]:
mt5.initialize()
mt5.login(LOGIN, PASSWORD, SERVER)

True

In [13]:
symbol = 'DE40'
timeframe = mt5.TIMEFRAME_M5
start_date = datetime(2022, 1, 1)
end_date = datetime(2022, 7, 1)
trade_volume = 0.5
commission = 1

In [14]:
bars = pd.DataFrame(mt5.copy_rates_range(symbol, timeframe, start_date, end_date))
bars['time'] = pd.to_datetime(bars['time'], unit='s')
bars['trade_volume'] = trade_volume
bars

KeyError: 'time'

In [ ]:
bars['sma_100'] = bars['close'].rolling(100).mean()
bars['sma_10'] = bars['close'].rolling(10).mean()
bars['hour'] = bars['time'].dt.hour
bars

In [ ]:
bars2 = bars.dropna().copy()

def get_signal(x):

    if 9 <= x['hour'] <= 18:

        if x['sma_10'] > x['sma_100']:
            return 1
        elif x['sma_10'] < x['sma_100']:
            return -1

    return 0

bars2['signal'] = bars2.apply(get_signal, axis=1)
bars2['prev_price'] = bars2['close'].shift(1)
bars2['price_change'] = bars2['close'] - bars2['prev_price']

bars2['signal_change'] = (bars2['signal'] - bars2['signal'].shift(1)).abs()
bars2['commission'] = bars2.apply(lambda x: commission * x['signal_change'] * x['trade_volume'], axis=1)
bars2

In [ ]:
bars3 = bars2.dropna().copy()
bars3['profit'] = bars3['signal'] * bars3['price_change'] * bars3['trade_volume']
bars3['gross_profit'] = bars3['profit'].cumsum()

bars3['net_profit'] = bars3['gross_profit'] - bars3['commission'].cumsum()


bars3

In [ ]:
px.line(bars3, 'time', ['gross_profit', 'net_profit'], title="DE40 Backtest MA Trendfollowing")